In [1]:
import json

import numpy as np
import pandas as pd
import pymongo
import scipy
from bson import ObjectId, json_util
from pandas.io.json import json_normalize
from pymongo import MongoClient as Connection
from scipy import sparse, spatial
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.metrics.pairwise import cosine_similarity

# scientific notation disabled form smaller numbers
pd.options.display.float_format = '{:.5f}'.format

# alles resultate anzeigen und nicht nur das letzte
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [2]:
# display multiple outputs in one row
import pandas as pd
import numpy as np
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [3]:
connection = Connection()
db = connection.recipe_db
input_data = db.recipes_without_reviews

data = json.loads(json_util.dumps(input_data.find()))

In [4]:
ing = pd.DataFrame(json_normalize(data, record_path='ingredients',
                             meta='id', record_prefix='ingredients_', errors='ignore'))


nutritions = pd.DataFrame(json_normalize(data, record_path='nutritions',
                            meta=['id', 'prep_time', 'rating', 'rating_count', 'ready_in_time', 'review_count']))


In [5]:
#  ------  erstellung und data cleansing - Ingredients

# schmeiss alle zutaten raus, die weniger als 5 mal verwendet werden
# setze multiinde auf 'id' und 'ingredients_id'
ingredients = ing.set_index(['id', 'ingredients_id'])

# filtere alle Zutaten samt ihrer rezepte id, die weniger gleich 5 mal vorkommen
ingredients_eqles_5_ing = ingredients.groupby(
    'ingredients_id').filter(lambda x: len(x) <= 5)

# droppe alle rezepte, die eine Zutate besitzen, die weniger gleich 5 Mal vorkommen
ingredients_filt = ingredients.drop(ingredients_eqles_5_ing.index.get_level_values('id').values, level=0)

# drop alls rows with ingredients_id == 0
ingredients_eqal_zero = ingredients_filt[ingredients_filt.index.get_level_values('ingredients_id') == 0]
ingredients_filt = ingredients_filt.drop(ingredients_eqal_zero.index.get_level_values('id').values, level=0)


In [6]:
#  ------ Erstellung und cleansing des Nutrition Dataframes

# erstelle neue liste auf basis der bereits gefilterten rezepte aus ingredients_filt
id_overlap_mask = nutritions['id'].isin(ingredients_filt.index.get_level_values('id').values)

# erstelle datenframe auf basis der overlapliste
nutritions_filt = nutritions.loc[id_overlap_mask]

nutrition_db = nutritions_filt.pivot_table(
    index=['id'],
    columns=['name'],
    values=['amount'],
).reset_index()

nutrition_db.set_index('id', inplace=True)

# remove multiindex 'amount'
nutrition_db.columns = nutrition_db.columns.droplevel(0)

# entferne alle NA
nutrition_db = nutrition_db.dropna()

# gleiche nochmals die ids der beiden dataframe nutrition und ingredients ab, da der nutrition dataframe noch NA Werte hatt
id_overlap_mask = ingredients_filt.index.get_level_values('id').isin(nutrition_db.index)
ingredients_db = ingredients_filt[id_overlap_mask]

# abgleich ob anzahl der indizes von nutrition und zutaten dataframe gleich sind


In [7]:


ingredients_db.reset_index(inplace=True)

recipe_db = pd.get_dummies(ingredients_db['ingredients_id']).groupby(
    ingredients_db['id']).apply(max)



In [8]:
new_ingredients = ingredients_db.copy()
#new_ingredients = new_ingredients.groupby("id")["ingredients_grams"].sum().reset_index()
gramms_ingredients = new_ingredients.groupby("id")["ingredients_grams"].sum().reset_index().copy()

Q1 = gramms_ingredients.quantile(0.25)
Q3 = gramms_ingredients.quantile(0.75)
IQR = Q3 - Q1

#Filter out all recipes which are outlier by their weight (gramms)
df = gramms_ingredients[~((gramms_ingredients < (Q1 - 1.5 * IQR)) |(gramms_ingredients > (Q3 + 1.5 * IQR))).any(axis=1)].copy()

#filter out recipes by weight which are not in the range 500 - 2373.59 gramms
df_start_at_fivehundret = df[df['ingredients_grams'].between(500, 2373.58225, inclusive=False)].copy()

df_start_at_fivehundret.set_index('id', inplace=True)
id_overlap_mask = nutritions['id'].isin(df_start_at_fivehundret.index.get_level_values('id').values)


# erstelle datenframe auf basis der overlapliste
nutritions_filt_gramm = nutritions.loc[id_overlap_mask]

nutrition_db2 = nutritions_filt_gramm.pivot_table(
    index=['id'],
    columns=['name'],
    values=['amount'],
).reset_index()

#create new nutrition db based on the above filtering
nutrition_db2.set_index('id', inplace=True)
nutrition_db2.columns = nutrition_db2.columns.droplevel(0)



In [9]:
# random
subset_random_top10 = nutrition_db2.sample(n=10).copy()
subset_random_top10.index.to_numpy()


array([ 54501, 214733,  11710,   8691, 215285, 220130,  25150, 159188,
        57532,  87421])

In [10]:
erg_2_normal_10_naiv = [223156, 191199, 107586,  14526, 231381,  48546,  57716,  24743, 233975, 9025]

In [11]:

subset_normal_id = erg_2_normal_10_naiv

In [12]:
# remove recipes which are missing in nutrition df from the ingredients tecipes dataframe
mask = recipe_db.index.isin(nutrition_db2.index.values)
recipe_db = recipe_db.loc[mask]


In [13]:
user_recipes_normal = recipe_db[recipe_db.index.isin(subset_normal_id)]

In [14]:
#top10 most common ingredients
#16421    2125 salt
#4342     1506 garlic
#4397     1412 onion
#16406    1185 ground black pepper
#16157    1016 butter
#6307      944 olive oil
#6494      786 skinless bone less chicken breast
#2496      749 water
#16238     574 grated Parmesan cheese
#16317     538 eggs


drop_id_list = [16421, 4342, 4397, 16406, 16157, 6307, 6494, 2496, 16238, 16317]





In [15]:
#filter out ingredients which are too common and the recipes in the user profile from the recipes database
new_recipe_db = recipe_db.drop(axis=1, labels=drop_id_list)


new_recipe_db_wo_userrecipes_normal = new_recipe_db.drop(axis=0, labels=subset_normal_id)


In [16]:
#remove basic ingredients like garlic, olive oil, skinless bone less chicken breast, butter, milk, salt and pepper, onion
new_user_recipes_normal = user_recipes_normal.drop(axis=1, labels=drop_id_list)

In [17]:
sample_normal = new_user_recipes_normal.sample(n=10).copy()

In [18]:
## Jaccard - normal

result_array = cdist(sample_normal, new_recipe_db_wo_userrecipes_normal,'jaccard')
result_w_filter_normal = pd.DataFrame(result_array, columns=new_recipe_db_wo_userrecipes_normal.index.values, index=sample_normal.index.values)
# result_2

result_w_filter_10_normal = pd.DataFrame(result_w_filter_normal.sum(), columns=['jaccard_distance_sum'])
result_w_filter_10_normal = result_w_filter_10_normal.sort_values(by='jaccard_distance_sum')
result_w_filter_10_normal['jaccard_distance_sum'] = result_w_filter_10_normal['jaccard_distance_sum'].div(10)
#result_w_filter_10_normal[0:10]


In [19]:
#create one dimensional ingredients vector from user recipes profile
user_one_vector_normal = sample_normal.copy()
user_one_vector_normal.loc['sum',:]= user_one_vector_normal.sum(axis=0)
user_one_vector_normal = user_one_vector_normal.drop(axis=0, labels=sample_normal.index)
user_one_vector_normal[user_one_vector_normal > 0] = 1

In [20]:
#user_one_vector_normal
#user_one_vector_normal.apply(pd.value_counts).count(axis=1)

In [21]:
## Jaccard - Vector normal top 10

result_array = cdist(user_one_vector_normal, new_recipe_db_wo_userrecipes_normal,'jaccard')
result_w_filter_vector_normal = pd.DataFrame(result_array, columns=new_recipe_db_wo_userrecipes_normal.index.values, index=user_one_vector_normal.index.values)


result_w_filter_vector_10_normal = pd.DataFrame(result_w_filter_vector_normal.sum(), columns=['jaccard_distance_sum'])
result_w_filter_vector_10_normal = result_w_filter_vector_10_normal.sort_values(by='jaccard_distance_sum')

#result_w_filter_vector_10_normal[0:10]


In [22]:
#result_w_filter_vector_10_normal[0:10].index.to_numpy()

In [23]:
####### results

In [24]:
# userrecipes normal
sample_normal.index.to_numpy()

array([191199,  14526, 233975,  24743, 107586, 223156,   9025, 231381,
        48546,  57716])

In [25]:
###### jaccard normal top 10 naiv
result_w_filter_10_normal[0:10].index.to_numpy()

array([143082, 201849, 241165,  20415,  40574, 141716, 162807, 240708,
       191156,  16862])

In [26]:
###### jaccard vector normal top 10
result_w_filter_vector_10_normal[0:10].index.to_numpy()

array([237807, 223101,  51039, 191156, 231230, 223100, 240708, 241601,
        77515,  20415])

In [27]:
##### random top 10
subset_random_top10.index.to_numpy()

array([ 54501, 214733,  11710,   8691, 215285, 220130,  25150, 159188,
        57532,  87421])

In [28]:
nutrition_db2.describe()

name,Calcium,Calories,Calories from Fat,Carbohydrates,Cholesterol,Dietary Fiber,Fat,Folate,Iron,Magnesium,Niacin Equivalents,Potassium,Protein,Saturated Fat,Sodium,Sugars,Thiamin,Vitamin A - IU,Vitamin B6,Vitamin C
count,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000,3770.00000
mean,147.62368,437.40977,206.09237,29.13842,97.12707,2.59778,22.89915,60.51086,3.15299,51.41984,12.44257,532.01939,28.05297,8.74708,851.17479,6.72738,0.30421,1168.79318,0.50982,16.28622
std,159.99250,183.07685,125.83504,22.82480,60.47892,2.50979,13.98167,59.87557,2.44330,29.10865,6.89345,267.47451,12.01118,6.79965,643.53732,8.81384,0.30164,2321.84247,0.31210,24.35803
min,2.24700,51.34326,1.32300,0.00000,0.00000,0.00000,0.14700,0.00000,0.19784,0.99745,0.03511,21.35641,0.75050,0.01992,5.20206,0.00000,0.00055,0.00000,0.00497,0.00000
25%,41.89560,302.98010,116.53312,10.22884,60.39150,0.74548,12.94812,15.88195,1.81476,33.22880,7.40300,345.45530,19.97516,3.62944,418.67972,1.76526,0.10510,236.94223,0.28412,2.15411
50%,83.16640,407.19045,182.75790,24.78570,85.15604,1.94681,20.30644,38.70213,2.76656,45.93100,11.15695,480.73480,26.86586,7.16939,734.11100,3.90747,0.19759,581.03190,0.44613,6.66003
75%,193.59578,541.44173,271.86345,43.25593,126.76047,3.66756,30.20705,88.07055,3.93232,63.26032,16.56040,654.20570,34.31604,12.13675,1092.85675,7.83992,0.39693,1113.54200,0.68317,19.56792
max,1264.32600,1828.19200,989.10300,236.72050,708.85000,27.83175,109.90030,519.53590,55.89075,590.39220,47.57578,1931.48800,99.72639,59.31002,7322.28900,104.81810,2.35457,38664.70000,3.83736,270.41310


In [29]:
#https://www.ncbi.nlm.nih.gov/books/NBK56068/table/summarytables.t4/?report=objectonly
#https://www.ncbi.nlm.nih.gov/books/NBK56068/table/summarytables.t1/?report=objectonly
#https://www.ncbi.nlm.nih.gov/books/NBK56068/table/summarytables.t5/?report=objectonly
#männer alter 31 - 50 pro tag geteil durch 3 mahlzeiten
#angepasst an einen keto (low/carb) dietplan https://www.healthline.com/nutrition/keto-diet-meal-plan-and-menu#bottom-line


recommenden_nut_low_carb = pd.DataFrame(data=None, columns=nutrition_db2.columns, index=None)


recommenden_nut_low_carb.loc['index'] = [
    # "Calcium":[1],
    266.67,
    # "Calories":[1],
    716,
    # "Calories from Fat":[1],
    0,
    # "Carbohydrates":[1],
    10,
    # "Cholesterol":[1],
    0,
    # "Dietary Fiber	":[1],
    12.67,
    # "Fat":[1],16.67,
    41.3,
    # "Folate":[1],
    133.3, 
    # "Iron":[1],
    2,
    # "Magnesium":[1],
    116.67, 
    # "Niacin Equivalents":[1],
    4,
    # "Potassium":[1],
    1,
    # "Protein":[1],
    53.3,
    # "Saturated Fat":[1],
    1,
    # "Sodium":[1],
    1,
    # "Sugars":[1],
    1,
    # "Thiamin":[1],
    0.3, 
    # "Vitamin A - IU":[1],
    208, 
    # "Vitamin B6":[1],
    0.36, 
    # "Vitamin C":[1]}
    25] 

In [130]:
no_weight = [
    # "Calcium":[1],
    1,
    # "Calories":[1],
    1,
    # "Calories from Fat":[1],
    1,
    # "Carbohydrates":[1],
    1,
    # "Cholesterol":[1],
    1,
    # "Dietary Fiber	":[1],
    1,
    # "Fat":[1],
    1,
    # "Folate":[1],
    1,
    # "Iron":[1],
    1,
    # "Magnesium":[1],
    1,
    # "Niacin Equivalents":[1],
    1,
    # "Potassium":[1],
    1,
    # "Protein":[1],
    1,
    # "Saturated Fat":[1],
    1,
    # "Sodium":[1],
    1,
    # "Sugars":[1],
    1,
    # "Thiamin":[1],
    1,
    # "Vitamin A - IU":[1],
    1,
    # "Vitamin B6":[1],
    1,
    # "Vitamin C":[1]}
    1]

weighted_carbs = [
    # "Calcium":[1],
    1,
    # "Calories":[1],
    10,
    # "Calories from Fat":[1],
    1,
    # "Carbohydrates":[1],
    1000,
    # "Cholesterol":[1],
    1,
    # "Dietary Fiber	":[1],
    1,
    # "Fat":[1],
    10,
    # "Folate":[1],
    1,
    # "Iron":[1],
    1,
    # "Magnesium":[1],
    1,
    # "Niacin Equivalents":[1],
    1,
    # "Potassium":[1],
    1,
    # "Protein":[1],
    10,
    # "Saturated Fat":[1],
    1,
    # "Sodium":[1],
    1,
    # "Sugars":[1],
    1,
    # "Thiamin":[1],
    1,
    # "Vitamin A - IU":[1],
    1,
    # "Vitamin B6":[1],
    1,
    # "Vitamin C":[1]}
    1]



In [241]:
result_w_filter_normal_100 = result_w_filter_10_normal[0:500].copy()
result_w_filter_normal_100['pos'] = np.arange(len(result_w_filter_normal_100))
result_w_filter_normal_100 = result_w_filter_normal_100.reset_index()
result_w_filter_normal_100 = result_w_filter_normal_100.set_index('index')

In [242]:
nutrition_normal_naiv = nutrition_db2[nutrition_db2.index.isin(result_w_filter_normal_100.index.to_numpy())]

In [243]:
# euklidische distanz für nährwerte

result_array = cdist(recommenden_nut_low_carb, nutrition_normal_naiv, 'minkowski', p=2, w=weighted_carbs)
euclid_distance = pd.DataFrame(
    result_array, columns=nutrition_normal_naiv.index.values, index=recommenden_nut_low_carb.index.values)

euclid_distance_sum_normal_naiv = pd.DataFrame(euclid_distance.sum(), columns=['euclid_distance_sum'])
euclid_distance_sum_normal_naiv = euclid_distance_sum_normal_naiv.sort_values(by='euclid_distance_sum')
euclid_distance_sum_normal_naiv[0:10]

,euclid_distance_sum
87421,832.56767
166292,921.11950
83792,925.04992
152156,937.06198
15395,985.29412
15005,1005.36882
107487,1010.00295
21752,1092.24503
150292,1106.26659
25475,1154.24545


In [244]:
jaccard_euclid_joined_normal_naiv = pd.merge(euclid_distance_sum_normal_naiv, result_w_filter_normal_100, left_index=True, right_index=True)

In [245]:
jaccard_euclid_joined_normal_naiv[0:10]

,euclid_distance_sum,jaccard_distance_sum,pos
87421,832.56767,0.95482,488
166292,921.11950,0.94134,393
83792,925.04992,0.89534,140
152156,937.06198,0.91147,221
15395,985.29412,0.95488,490
15005,1005.36882,0.87729,65
107487,1010.00295,0.94591,421
21752,1092.24503,0.94579,417
150292,1106.26659,0.93069,340
25475,1154.24545,0.92901,327


In [246]:
nutrition_top10_normal_naiv = nutrition_db2[nutrition_db2.index.isin(jaccard_euclid_joined_normal_naiv[0:10].index.to_numpy())]

In [247]:
nutrition_top10_normal_naiv = nutrition_top10_normal_naiv[['Calories', 'Fat', 'Protein', 'Carbohydrates', 'Dietary Fiber', 'Iron', 'Calcium', 'Magnesium' ]]
nutrition_top10_normal_naiv.describe()

name,Calories,Fat,Protein,Carbohydrates,Dietary Fiber,Iron,Calcium,Magnesium
count,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000
mean,593.49335,43.66549,36.49309,10.57661,0.54316,2.16690,41.57485,37.28615
std,59.98479,11.32938,11.34957,6.51885,0.51290,1.06616,22.77249,17.53916
min,514.51740,29.74562,14.67175,1.95229,0.00000,0.59953,17.25907,7.52194
25%,563.72928,40.36646,32.43579,7.74047,0.18195,1.31123,25.73577,22.45584
50%,572.76435,40.73643,38.89960,9.12031,0.33888,2.30381,35.67709,41.11308
75%,611.76150,44.27074,43.73848,11.94262,0.95037,2.96299,45.03080,48.91684
max,726.85720,71.04302,51.30702,27.00220,1.32867,3.78598,88.56937,59.62160


In [248]:
nutrition_top10_normal_naiv

name,Calories,Fat,Protein,Carbohydrates,Dietary Fiber,Iron,Calcium,Magnesium
id,,,,,,,,
15005,620.31840,41.05270,51.30702,8.72020,0.33416,2.71324,37.85680,59.62160
15395,514.51740,34.18698,42.24757,6.81862,0.00197,2.09525,31.02137,46.27038
21752,561.08360,29.74562,33.58570,27.00220,0.00000,0.81046,46.39300,49.79900
25475,654.22680,45.06601,44.23545,12.40110,0.29570,2.51237,73.19752,56.42002
83792,726.85720,71.04302,14.67175,8.87978,0.14404,1.13887,23.97390,17.69848
87421,571.74400,41.88495,32.05249,10.94046,1.06261,3.78598,33.49738,35.04420
107487,573.78470,40.42016,39.15875,12.27667,1.30722,3.13876,88.56937,41.57553
150292,586.09080,40.35315,47.22263,9.36084,1.32867,0.59953,17.25907,7.52194
152156,554.64430,40.40639,38.64046,7.41389,0.61368,3.04624,40.94420,40.65062


In [249]:
nutrition_top10_normal_naiv.index.to_numpy()

array([ 15005,  15395,  21752,  25475,  83792,  87421, 107487, 150292,
       152156, 166292])

In [250]:
nutrition_normal_naiv_copy = nutrition_normal_naiv[['Calories', 'Fat', 'Protein', 'Carbohydrates', 'Dietary Fiber', 'Iron', 'Calcium', 'Magnesium' ]]

In [251]:
nutrition_normal_naiv_copy.describe()

name,Calories,Fat,Protein,Carbohydrates,Dietary Fiber,Iron,Calcium,Magnesium
count,500.00000,500.00000,500.00000,500.00000,500.00000,500.00000,500.00000,500.00000
mean,379.91200,18.77473,27.77814,24.38722,1.67891,2.64592,65.16221,49.14609
std,165.96359,13.16671,11.39625,20.78219,1.81817,1.46535,88.27415,25.55334
min,77.87188,1.09763,1.25905,0.85675,0.00000,0.39677,7.72080,6.70075
25%,258.64975,9.32379,20.14825,9.65446,0.43949,1.61578,30.68406,32.72087
50%,356.15760,15.71317,25.93048,18.56389,0.97620,2.37335,43.55357,42.95623
75%,455.64660,24.77847,33.33295,33.35638,2.46734,3.37176,66.05542,58.13152
max,1218.62000,97.29582,71.10712,140.09960,13.80218,11.13676,841.94360,179.86410


In [252]:
nutrition_random_10 = nutrition_db2[nutrition_db2.index.isin(new_recipe_db_wo_userrecipes_normal.sample(n=10).index.to_numpy())]

In [253]:
nutrition_random_10 = nutrition_random_10[['Calories', 'Fat', 'Protein', 'Carbohydrates', 'Dietary Fiber', 'Iron', 'Calcium', 'Magnesium' ]]
nutrition_random_10.describe()

name,Calories,Fat,Protein,Carbohydrates,Dietary Fiber,Iron,Calcium,Magnesium
count,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000
mean,372.85338,17.18158,22.35458,32.19674,2.86552,2.80277,141.30790,58.07154
std,181.22307,9.11458,8.89835,42.20319,2.36445,1.08010,128.53617,33.73797
min,137.17080,1.11104,5.86232,2.17963,0.10143,1.36694,17.56674,19.52003
25%,282.29440,11.53061,20.84503,7.26532,0.98527,1.79282,26.97501,29.67385
50%,315.55170,17.63027,24.59386,14.18698,2.30382,3.06877,115.94311,53.09722
75%,484.21037,20.80701,27.17541,40.47207,4.27600,3.38526,242.24013,76.63119
max,757.51500,34.86615,31.89189,140.09960,7.32798,4.76431,364.76480,109.09760


In [254]:
nutrition_jaccard_normal = nutrition_db2[nutrition_db2.index.isin(result_w_filter_10_normal[0:10].index.to_numpy())]

In [255]:
nutrition_jaccard_normal = nutrition_jaccard_normal[['Calories', 'Fat', 'Protein', 'Carbohydrates', 'Dietary Fiber', 'Iron', 'Calcium', 'Magnesium' ]]
nutrition_jaccard_normal.describe()

name,Calories,Fat,Protein,Carbohydrates,Dietary Fiber,Iron,Calcium,Magnesium
count,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000
mean,377.88154,18.15332,27.52207,26.34726,1.94061,2.62302,60.55281,52.68198
std,142.98780,14.57187,10.59497,13.83804,1.48313,1.14005,34.29074,26.59647
min,202.86680,4.15611,15.71891,13.29307,0.21804,1.07068,20.02205,25.87718
25%,320.61210,10.01874,21.96832,16.84179,0.67652,1.66299,34.09878,34.31395
50%,343.71595,13.92654,26.04003,20.15478,1.79942,2.66837,57.26450,45.50204
75%,433.65340,20.97736,29.15491,34.79741,3.22858,3.39623,74.09853,58.82804
max,709.98490,55.53167,53.39647,54.69534,4.13308,4.22336,129.81980,114.62010


In [256]:
nutrition_user_recipes_normal = nutrition_db2[nutrition_db2.index.isin(user_recipes_normal.index.to_numpy())]

In [257]:
nutrition_user_recipes_normal = nutrition_user_recipes_normal[['Calories', 'Fat', 'Protein', 'Carbohydrates', 'Dietary Fiber', 'Iron', 'Calcium', 'Magnesium' ]]

In [258]:
nutrition_user_recipes_normal.describe()

name,Calories,Fat,Protein,Carbohydrates,Dietary Fiber,Iron,Calcium,Magnesium
count,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000,10.00000
mean,373.20160,17.43535,27.38771,25.02173,2.01970,2.92550,51.04455,50.53929
std,167.68859,13.09345,14.66362,21.86542,2.77605,1.23483,18.74117,30.30378
min,159.87500,1.51186,6.72020,5.49834,0.11800,0.95962,19.36939,19.33667
25%,281.49715,9.74220,19.87937,13.55111,0.26699,2.27362,47.41096,29.47750
50%,311.05070,16.33429,23.46711,14.33521,0.97516,3.17336,51.84065,42.14446
75%,490.08047,19.43150,28.54940,28.41936,2.76950,3.40366,63.07875,57.97394
max,652.74430,44.00749,56.59666,72.76235,9.08962,5.50868,80.20887,118.93400
